In [14]:
from google.cloud import bigquery
from google.cloud.bigquery import job
from google.cloud.bigquery import SchemaField
import pandas as pd
import datetime as dt
import numpy as np
import re
import os
from tqdm import tqdm
import pandas as pd
PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)

In [2]:
# 클라이언트 설정
client = bigquery.Client()
query_name = "0711_동적쿼리테스트"
# SQL 파일 읽기
# file = './query/쿠팡0403_0509.sql'
file = f'../query/{query_name}.sql'
with open(file, 'r') as file:
    query = file.read()


In [3]:

# 쿼리 실행
job_config = bigquery.QueryJobConfig()
query_job = client.query(query, job_config=job_config)

df = query_job.to_dataframe()

In [4]:
df.loc[0]['e'][0]['d'][0]['products_struct_alias'] = []
df.loc[0]['e'][0]['d'][1]['products_struct_alias'] = []

In [5]:
def infer_schema_from_dataframe(df):
    """데이터프레임에서 스키마 추론."""
    def infer_field_type(value):
        if isinstance(value, dict):
            subfields = {k: infer_field_type(v) for k, v in value.items()}
            return {"type": "RECORD", "fields": subfields}
        elif isinstance(value, (np.ndarray, list)) and len(value) > 0 and isinstance(value[0], dict):
            subfields = {k: infer_field_type(v) for k, v in value[0].items()}
            return {"type": "RECORD", "mode": "REPEATED", "fields": subfields}
        elif isinstance(value, (np.ndarray, list)):
            return {"type": "STRING", "mode": "REPEATED"}
        elif pd.api.types.is_integer_dtype(type(value)):
            return "INTEGER"
        elif pd.api.types.is_float_dtype(type(value)):
            return "FLOAT"
        elif pd.api.types.is_bool_dtype(type(value)):
            return "BOOLEAN"
        elif pd.api.types.is_datetime64_any_dtype(type(value)):
            return "TIMESTAMP"
        else:
            return "STRING"
    
    schema = {col: infer_field_type(df[col].iloc[0]) for col in df.columns}

    return schema


infer_schema_from_dataframe(df)


{'Event_Date': 'STRING',
 'Airbridge_Device_ID': 'STRING',
 'Airbridge_Device_ID_Type': 'STRING',
 'i': {'type': 'RECORD',
  'fields': {'Device_Model': 'STRING',
   'Device_Type': 'STRING',
   'Platform': 'STRING',
   'Client_IP_Country_Code': 'STRING',
   'Client_IP_Subdivision': 'STRING',
   'Client_IP_City': 'STRING',
   'Campaign_ID': 'STRING',
   'Ad_Group_ID': 'STRING',
   'Ad_Creative_ID': 'STRING',
   'Term_ID': 'STRING',
   'Is_Re_engagement': 'BOOLEAN',
   'Is_First_Event_per_User_ID': 'BOOLEAN',
   'Is_First_Event_per_Device_ID': 'BOOLEAN',
   'Is_First_Target_Event_per_Device': 'BOOLEAN',
   'Target_Event_Timestamp': 'STRING',
   'Target_Event_Category': 'STRING'}},
 'Event_Category': 'STRING',
 'Event_Value_Total': 'FLOAT',
 'Event_Count_Total': 'INTEGER',
 'e': {'type': 'RECORD',
  'mode': 'REPEATED',
  'fields': {'Label': 'STRING',
   'Action': 'STRING',
   'Event_Value_Sum': 'FLOAT',
   'Event_Count': 'INTEGER',
   'u': {'type': 'RECORD',
    'mode': 'REPEATED',
    'fi

In [66]:

df[df.columns[-1]].iloc[0]

array([{'Label': '상품상세_구매하기', 'Action': '동서)스타벅스커피라떼컵200ml', 'Event_Value_Sum': 6000.0, 'Event_Count': 2, 'u': array([{'User_ID': '04432efd-e80f-4800-a0ba-ef4564abf3e2'},
              {'User_ID': '04432efd-e80f-4800-a0ba-ef4564abf3e2'}], dtype=object), 'd': array([{'Timestamp': '2024-07-11T15:57:48+09:00', 'Label': '상품상세_구매하기', 'Action': '동서)스타벅스커피라떼컵200ml', 'Value': 3000.0, 'name': None, 'price': None, 'productID': None, 'transactionID': None, '_field_9': None, 'tester': 'true', 'products_struct_alias': []},
              {'Timestamp': '2024-07-11T15:58:12+09:00', 'Label': '상품상세_구매하기', 'Action': '동서)스타벅스커피라떼컵200ml', 'Value': 3000.0, 'name': None, 'price': None, 'productID': None, 'transactionID': None, '_field_9': None, 'tester': 'true', 'products_struct_alias': []}],
             dtype=object)}                                                                                                                                                                                                

In [192]:
df.loc[0]['i']

{'Device_Model': 'SM-N971N',
 'Device_Type': 'mobile',
 'Platform': 'Android',
 'Client_IP_Country_Code': 'KR',
 'Client_IP_Subdivision': 'Gyeonggi-do',
 'Client_IP_City': 'Suwon',
 'Campaign_ID': None,
 'Ad_Group_ID': None,
 'Ad_Creative_ID': None,
 'Term_ID': None,
 'Is_Re_engagement': True,
 'Is_First_Event_per_User_ID': False,
 'Is_First_Event_per_Device_ID': False,
 'Is_First_Target_Event_per_Device': False,
 'Target_Event_Timestamp': Decimal('1720536693400.000000000'),
 'Target_Event_Category': 'Deeplink Open (App)'}

In [55]:
df[df.columns[-1]].iloc[0]

array([{'Label': '상품상세_구매하기', 'Action': '동서)스타벅스커피라떼컵200ml', 'Event_Value_Sum': 6000.0, 'Event_Count': 2, 'u': array([{'User_ID': '04432efd-e80f-4800-a0ba-ef4564abf3e2'},
              {'User_ID': '04432efd-e80f-4800-a0ba-ef4564abf3e2'}], dtype=object), 'd': array([{'Timestamp': '2024-07-11T15:57:48+09:00', 'Label': '상품상세_구매하기', 'Action': '동서)스타벅스커피라떼컵200ml', 'Value': 3000.0, 'name': None, 'price': None, 'productID': None, 'transactionID': None, '_field_9': None, 'tester': 'true', 'products_struct_alias': []},
              {'Timestamp': '2024-07-11T15:58:12+09:00', 'Label': '상품상세_구매하기', 'Action': '동서)스타벅스커피라떼컵200ml', 'Value': 3000.0, 'name': None, 'price': None, 'productID': None, 'transactionID': None, '_field_9': None, 'tester': 'true', 'products_struct_alias': []}],
             dtype=object)}                                                                                                                                                                                                

전체 스키마 조회 및 승자독식 구현

In [6]:

df['Event_Date'] = pd.to_datetime(df.Event_Date)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1476 entries, 0 to 1475
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Event_Date                1476 non-null   datetime64[ns]
 1   Airbridge_Device_ID       1476 non-null   object        
 2   Airbridge_Device_ID_Type  1476 non-null   object        
 3   i                         1476 non-null   object        
 4   Event_Category            1476 non-null   object        
 5   Event_Value_Total         1476 non-null   float64       
 6   Event_Count_Total         1476 non-null   Int64         
 7   e                         1476 non-null   object        
dtypes: Int64(1), datetime64[ns](1), float64(1), object(5)
memory usage: 93.8+ KB


In [7]:
df.loc[0]['i']

{'Device_Model': 'SM-N971N',
 'Device_Type': 'mobile',
 'Platform': 'Android',
 'Client_IP_Country_Code': 'KR',
 'Client_IP_Subdivision': 'Gyeonggi-do',
 'Client_IP_City': 'Suwon',
 'Campaign_ID': None,
 'Ad_Group_ID': None,
 'Ad_Creative_ID': None,
 'Term_ID': None,
 'Is_Re_engagement': True,
 'Is_First_Event_per_User_ID': False,
 'Is_First_Event_per_Device_ID': False,
 'Is_First_Target_Event_per_Device': False,
 'Target_Event_Timestamp': Decimal('1720536693400.000000000'),
 'Target_Event_Category': 'Deeplink Open (App)'}

In [8]:
pd.api.types.is_bool_dtype(type(df.loc[0]['i']['Is_First_Target_Event_per_Device']))
# df.loc[0]['i']

True

In [4]:
def infer_schema_from_dataframe(df):
    """데이터프레임에서 스키마 추론."""
    def infer_field_type(value):
        if isinstance(value, dict):
            subfields = {k: infer_field_type(v) for k, v in value.items()}
            return {"type": "RECORD", "fields": subfields}
        elif isinstance(value, (np.ndarray, list)) and len(value) > 0 and isinstance(value[0], dict):
            subfields = {k: infer_field_type(v) for k, v in value[0].items()}
            return {"type": "RECORD", "mode": "REPEATED", "fields": subfields}
        elif isinstance(value, (np.ndarray, list)):
            return {"type": "STRING", "mode": "REPEATED"}
        elif pd.api.types.is_integer_dtype(type(value)):
            return "INTEGER"
        elif pd.api.types.is_float_dtype(type(value)):
            return "FLOAT"
        elif pd.api.types.is_bool_dtype(type(value)):
            return "BOOLEAN"
        elif pd.api.types.is_datetime64_any_dtype(type(value)):
            return "TIMESTAMP"
        elif isinstance(value, pd.Timestamp):
            return "DATE"
        else:
            return "STRING"
    
    schema = {}
    
    for row in range(len(df)):
        post_schema = {col: infer_field_type(df[col].iloc[row]) for col in df.columns}        
        schema = merge_schemas(schema, post_schema)

    return schema


def order_schema(schema, order):
    ordered_schema = {}
    for key in order:
        if key in schema:
            ordered_schema[key] = schema[key]
    return ordered_schema


In [5]:
def merge_schemas(schema1, schema2):
    if isinstance(schema1, dict) and isinstance(schema2, dict):
        merged = {}
        for key in set(schema1.keys()).union(set(schema2.keys())):
            if key in schema1 and key in schema2:
                if schema1[key] == schema2[key]:
                    merged[key] = schema1[key]
                else:
                    merged[key] = merge_schemas(schema1[key], schema2[key])
            elif key in schema1:
                merged[key] = schema1[key]
            elif key in schema2:
                merged[key] = schema2[key]
        return merged
    else:
        return schema1 if schema1 == schema2 else "STRING"

In [6]:
def convert_to_schema_fields(schema):
    """스키마 딕셔너리를 SchemaField 객체로 변환."""
    fields = []
    for name, dtype in schema.items():
        if isinstance(dtype, dict):
            subfields = convert_to_schema_fields(dtype.get("fields", {}))
            mode = dtype.get("mode", "NULLABLE")
            fields.append(SchemaField(name, dtype["type"], mode=mode, fields=subfields))
        else:
            fields.append(SchemaField(name, dtype))
    return fields


In [7]:
def get_current_schema(table_ref):
    """기존 테이블의 스키마를 가져옵니다."""
    table = client.get_table(table_ref)
    # return {field.name: field for field in table.schema}
    return table.schema
    
def update_table_schema(table_ref, new_columns):
    """테이블 스키마에 새 열을 추가합니다."""
    current_schema = get_current_schema(table_ref)
    new_fields = []

    for name, dtype in new_columns.items():
        if name not in current_schema:
            if isinstance(dtype, dict):
                new_fields.extend(convert_to_schema_fields({name: dtype}))
            else:
                new_fields.append(SchemaField(name, dtype))

    if new_fields:
        table = client.get_table(table_ref)
        updated_schema = table.schema + new_fields
        table.schema = updated_schema
        client.update_table(table, ["schema"])
        print(f"테이블 {table_ref}의 스키마가 업데이트되었습니다: {new_columns}")

def load_data_to_bigquery(df, table_ref):
    """데이터프레임을 빅쿼리 테이블로 업로드합니다."""
    job = client.load_table_from_dataframe(df, table_ref)
    job.result()
    print(f"테이블 {table_ref}에 데이터가 성공적으로 업로드되었습니다.")

In [8]:
def update_table_schema(table_ref, new_columns):
    """테이블 스키마에 새 열을 추가합니다."""
    client = bigquery.Client()
    table = client.get_table(table_ref)
    current_schema = {field.name: field for field in table.schema}
    updated_schema = list(table.schema)  # 기존 스키마 유지

    def add_new_fields(new_fields, parent_fields):
        parent_fields_dict = {field.name: field for field in parent_fields}
        for name, dtype in new_fields.items():
            if name not in parent_fields_dict:
                if isinstance(dtype, dict):
                    subfields = convert_to_schema_fields(dtype.get("fields", {}))
                    mode = dtype.get("mode", "NULLABLE")
                    parent_fields.append(SchemaField(name, dtype["type"], mode=mode, fields=subfields))
                else:
                    parent_fields.append(SchemaField(name, dtype))
            else:
                existing_field = parent_fields_dict[name]
                if existing_field.field_type == 'RECORD' and 'fields' in dtype:
                    existing_field_fields_list = list(existing_field.fields)
                    add_new_fields(dtype['fields'], existing_field_fields_list)
                    parent_fields[parent_fields.index(existing_field)] = SchemaField(
                        name=existing_field.name,
                        field_type=existing_field.field_type,
                        mode=existing_field.mode,
                        fields=tuple(existing_field_fields_list)
                    )

    add_new_fields(new_columns, updated_schema)

    table.schema = updated_schema
    client.update_table(table, ["schema"])
    print(f"테이블 {table_ref}의 스키마가 업데이트되었습니다: {new_columns}")

In [10]:
# 기존 스키마 추출
def convert_schema_fields(schema):
    def convert_field(field):
        if field.field_type == 'RECORD':
            return {
                "type": field.field_type,
                "mode": field.mode,
                "fields": {subfield.name: convert_field(subfield) for subfield in field.fields}
            }
        elif field.field_type == 'STRING' and field.mode == 'REPEATED':
            return {
                "type": field.field_type,
                "mode": field.mode,
                "fields": {subfield.name: convert_field(subfield) for subfield in field.fields} if field.fields else {}
            }
        else:
            return field.field_type

    return {field.name: convert_field(field) for field in schema}

def find_keys(data, pattern):
    matches = []
    if isinstance(data, dict):
        for key, value in data.items():
            if re.match(pattern, key):
                matches.append(key)
            if isinstance(value, dict):
                matches.extend(find_keys(value, pattern))
            elif isinstance(value, list):
                for item in value:
                    matches.extend(find_keys(item, pattern))
    return matches

# tg_field = new_data['e']['fields']['d']

# pattern = r'^_field'
# matches = find_keys(tg_field, pattern)
# matches[0]

def replace_keys(data, pattern, replacement):
    if isinstance(data, dict):
        new_data = {}
        for key, value in data.items():
            new_key = re.sub(pattern, replacement, key)
            new_data[new_key] = replace_keys(value, pattern, replacement)
        return new_data
    elif isinstance(data, list):
        return [replace_keys(item, pattern, replacement) for item in data]
    else:
        return data


In [11]:
def load_data_to_bigquery(df, table_ref):
    """데이터프레임을 빅쿼리 테이블로 업로드합니다."""
    job = client.load_table_from_dataframe(df, table_ref)
    job.result()
    print(f"테이블 {table_ref}에 데이터가 성공적으로 업로드되었습니다.")

In [12]:
def pop_error_new_df(df,tg):
    tg = '_field_9'
    for index, row in df.iterrows():
        for e_item in row['e']:
            for d_item in e_item['d']:
                if tg in d_item:
                    d_item.pop(tg)


In [13]:
# 기존 데이터셋 수집
dataset_id = 'airbridge_mart'
table_id = 'app_df_2'
table_ref = f'ballosodeuk.{dataset_id}.{table_id}'


In [22]:
## --- 신규 데이터 스키마 --- ##
new_columns = infer_schema_from_dataframe(df)

## --- 신규 스키마 <> 기존 스키마 --- ##
current_schema = get_current_schema(table_ref)

existed_schema = convert_schema_fields(current_schema)
# 기존 스키마의 field 값 찾기
tg_field = existed_schema['e']['fields']['d']

pattern = r'^_field'
matches = find_keys(tg_field, pattern)
field_name = matches[0]

# 새로운 스키마 의 field 값 변경
pattern = r'^_field_(\d+)$'
# replacement = field_name
replacement = 'field_test'
print(replacement)

# new_data = replace_keys(new_columns, pattern, replacement)
# # print(new_data)

# # # --- 신규 스키마 정렬 및 bq 스키마화 -- ##
# schema = order_schema(new_data,df.columns)
# schema = convert_to_schema_fields(schema)

# print("-- 정렬 및 스키마 화 완료 --")

# # ## --- 테이블 스키마 Update --- ##
# update_table_schema(table_ref, new_data)

# # ## --- 데이터 업로드 --- ##
# # load_data_to_bigquery(df, table_ref)


'_field_9'

데이터프레임

In [253]:
def rename_specific_key(obj, old_key, new_key):
    if isinstance(obj, list):
        return [rename_specific_key(item, old_key, new_key) for item in obj]
    elif isinstance(obj, dict):
        return {new_key if k == old_key else k: rename_specific_key(v, old_key, new_key) for k, v in obj.items()}
    return obj

df['e'] = df['e'].apply(lambda arr: np.array([rename_specific_key(item, '_field_9', '_field_8') for item in arr], dtype=object))

df['e'][0]


array([{'Label': '상품상세_구매하기', 'Action': '동서)스타벅스커피라떼컵200ml', 'Event_Value_Sum': 6000.0, 'Event_Count': 2, 'u': array([{'User_ID': '04432efd-e80f-4800-a0ba-ef4564abf3e2'},
              {'User_ID': '04432efd-e80f-4800-a0ba-ef4564abf3e2'}], dtype=object), 'd': array([{'Timestamp': '2024-07-11T15:57:48+09:00', 'Label': '상품상세_구매하기', 'Action': '동서)스타벅스커피라떼컵200ml', 'Value': 3000.0, 'name': None, 'price': None, 'productID': None, 'transactionID': None, '_field_9': None, 'tester': 'true', 'products_struct_alias': []},
              {'Timestamp': '2024-07-11T15:58:12+09:00', 'Label': '상품상세_구매하기', 'Action': '동서)스타벅스커피라떼컵200ml', 'Value': 3000.0, 'name': None, 'price': None, 'productID': None, 'transactionID': None, '_field_9': None, 'tester': 'true', 'products_struct_alias': []}],
             dtype=object)}                                                                                                                                                                                                

In [ ]:
def find_keys(data, pattern):
    matches = []
    if isinstance(data, dict):
        for key, value in data.items():
            if re.match(pattern, key):
                matches.append(key)
            if isinstance(value, dict):
                matches.extend(find_keys(value, pattern))
            elif isinstance(value, list):
                for item in value:
                    matches.extend(find_keys(item, pattern))
    return matches

tg_field = new_data['e']['fields']['d']

pattern = r'^_field'
matches = find_keys(tg_field, pattern)
matches[0]

'_field_1'

In [276]:
df['e'][0][0]['d'][0].pop('_field_9', None)

In [278]:
df['e'][0][1]['d'][0]

{'Timestamp': '2024-07-11T15:34:46+09:00',
 'Label': '상품상세_구매하기',
 'Action': '제주 말차 크림 프라푸치노 T',
 'Value': 9000.0,
 'name': None,
 'price': None,
 'productID': None,
 'transactionID': None,
 '_field_9': None,
 'tester': 'true',
 'products_struct_alias': array([{'name': '제주 말차 크림 프라푸치노 T', 'price': '9000', 'position': None}],
       dtype=object)}

In [308]:
for index, row in df.iterrows():
    tg = len(df['e'][0][1]['d'])
    tg = len(row['e'][0]['d'])
    for i in range(tg):
        print(row['e'][i]['d'])

[{'Timestamp': '2024-07-11T15:57:48+09:00', 'Label': '상품상세_구매하기', 'Action': '동서)스타벅스커피라떼컵200ml', 'Value': 3000.0, 'name': None, 'price': None, 'productID': None, 'transactionID': None, 'tester': 'true', 'products_struct_alias': []}
 {'Timestamp': '2024-07-11T15:58:12+09:00', 'Label': '상품상세_구매하기', 'Action': '동서)스타벅스커피라떼컵200ml', 'Value': 3000.0, 'name': None, 'price': None, 'productID': None, 'transactionID': None, '_field_9': None, 'tester': 'true', 'products_struct_alias': []}]
[{'Timestamp': '2024-07-11T15:34:46+09:00', 'Label': '상품상세_구매하기', 'Action': '제주 말차 크림 프라푸치노 T', 'Value': 9000.0, 'name': None, 'price': None, 'productID': None, 'transactionID': None, '_field_9': None, 'tester': 'true', 'products_struct_alias': array([{'name': '제주 말차 크림 프라푸치노 T', 'price': '9000', 'position': None}],
       dtype=object)}                                                                                                                                                                                  

IndexError: index 1 is out of bounds for axis 0 with size 1

In [309]:
def pop_error_new_df(df,tg):
    tg = '_field_9'
    for index, row in df.iterrows():
        for e_item in row['e']:
            for d_item in e_item['d']:
                if tg in d_item:
                    d_item.pop(tg)



In [318]:
df.e[200][0]

{'Label': '상품상세_구매하기',
 'Action': '불싸이버거',
 'Event_Value_Sum': 6857.0,
 'Event_Count': 1,
 'u': array([{'User_ID': 'a2429cdd-b07f-4a91-9352-0cda4b956d39'}], dtype=object),
 'd': array([{'Timestamp': '2024-07-11T17:12:13+09:00', 'Label': '상품상세_구매하기', 'Action': '불싸이버거', 'Value': 6857.0, 'name': None, 'price': None, 'productID': None, 'transactionID': None, 'tester': 'true', 'products_struct_alias': array([{'name': '불싸이버거', 'price': '6857', 'position': None}], dtype=object)}],
       dtype=object)}

In [283]:
for i in len(df):
len(df['e'][0][1]['d'])

1

In [280]:
def remove_field_9(obj):
    if isinstance(obj, list):
        return [remove_field_9(item) for item in obj]
    elif isinstance(obj, dict):
        if '_field_9' in obj:
            obj.pop('_field_9', None)
        for k, v in obj.items():
            obj[k] = remove_field_9(v)
    return obj

# Apply the function to remove '_field_9' in the nested dictionaries within the DataFrame
df['e'] = df['e'].apply(lambda arr: np.array([remove_field_9(item) for item in arr], dtype=object))


df['e'][100]

array([{'Label': '상품상세_구매하기', 'Action': '서울)올데이프룻오렌지250ml', 'Event_Value_Sum': 6858.0, 'Event_Count': 3, 'u': array([{'User_ID': '6498c9f5-da63-4307-bff3-f601259bae4f'},
              {'User_ID': '6498c9f5-da63-4307-bff3-f601259bae4f'},
              {'User_ID': '6498c9f5-da63-4307-bff3-f601259bae4f'}], dtype=object), 'd': array([{'Timestamp': '2024-07-11T16:30:50+09:00', 'Label': '상품상세_구매하기', 'Action': '서울)올데이프룻오렌지250ml', 'Value': 2286.0, 'name': None, 'price': None, 'productID': None, 'transactionID': None, '_field_9': None, 'tester': 'true', 'products_struct_alias': array([{'name': '서울)올데이프룻오렌지250ml', 'price': '2286', 'position': None}],
                    dtype=object)}                                                                                                                                                                                                                                                                                                        ,
              {'Tim